In [24]:
from nmon import *

N = 1
M = 2
EC_shunt = 0.18 * 1

EJ_EC = 1

nmon = Nmon(N=N, M=M, EJN=EC_shunt*EJ_EC*20, EJM=EC_shunt*EJ_EC*20, EC_shunt=EC_shunt)
nmon.flag_calc_transitions = True
nmon.hamiltonian_calc(flux=0.0, ng=[0.0, 0.0], num_levels=6, make_plot=True, just_H=True, cutoff=7)
                    #    cutoff=compute_cutoff(nmon.EJN, nmon.EJM, nmon.EC_total))
# print(nmon.transition_freqs)
# print(nmon.bound_state_energies)
# print(nmon.relative_anharm)
print(nmon.sym_hamiltonian)

(-3.6*cos(θ1) - 3.6*cos((2πΦ_{1}) - θ2) - 3.6*cos(θ1 + θ2)) + (62.678458*n1**2 - 124.643084*n1*n2 + 125.356916*n1*n_g1 - 124.643084*n1*n_g2 + 62.678458*n2**2 - 124.643084*n2*n_g1 + 125.356916*n2*n_g2 + 62.678458*n_g1**2 - 124.643084*n_g1*n_g2 + 62.678458*n_g2**2)


In [28]:
import numpy as np
import sympy as sp

def construct_hamiltonian_from_sym(sym_hamiltonian, n_cut, Phi_1=0.0, n_g1=0.0, n_g2=0.0):
    """
    Build and diagonalize a 2-island Hamiltonian in the charge basis from a Sympy expression.
    
    sym_hamiltonian: Sympy expression involving n1, n2, n_g1, n_g2, (2πΦ_{1}), cos(theta1),
                     cos((2πΦ_{1}) - theta2), cos(theta1 + theta2), etc.
    n_cut: integer cutoff for each n1, n2 dimension
    Phi_1, n_g1, n_g2: external flux and offset charges
    
    Returns: (eigenvalues, eigenvectors, H_matrix)
    """
    # 1) Define symbolic variables (used in your circuit)
    n1_sym, n2_sym = sp.symbols('n1 n2', real=True)
    ng1_sym, ng2_sym = sp.symbols('n_g1 n_g2', real=True)
    # phi_sym = sp.symbols('(2πΦ_{1})', real=True)  # matches typical scqubits naming
    theta1_sym, theta2_sym = sp.symbols('θ1 θ2', real=True)

    # 1) The old symbol in scqubits might be:
    phi_sym_original = sp.Symbol('(2πΦ_{1})', real=True)
    # 2) Define a new symbol with a Python-friendly name:
    phi_sym = sp.Symbol('phi_1', real=True)
    # 3) xreplace in the Hamiltonian expression
    sym_hamiltonian = sym_hamiltonian.xreplace({phi_sym_original: phi_sym})


    # 2) Expand possible trig in the expression
    ham_expanded = sp.expand_trig(sym_hamiltonian)

    # 3) Separate out polynomial vs. trigonometric terms
    cos_terms = []
    polynomial_part = 0

    # sp.Add.make_args(...) splits the top-level sum.
    for term in sp.Add.make_args(ham_expanded):
        # Check if the term has Cos(...) (or Sin(...)) in it
        if term.has(sp.cos) or term.has(sp.sin):
            cos_terms.append(term)
        else:
            polynomial_part += term

    # 4) Lambdify the polynomial part in terms of (n1, n2, n_g1, n_g2)
    poly_func = sp.lambdify((n1_sym, n2_sym, ng1_sym, ng2_sym), polynomial_part, 'numpy')

    # 5) For each cos(...) term, store (coefficient_function, cos_argument).
    cos_data_list = []
    for cterm in cos_terms:
        # We'll look for the actual cos(...) object. 
        # If your Hamiltonian only has cos(...), we can do:
        cos_objs = list(cterm.atoms(sp.cos))
        if not cos_objs:
            # If there's also sin(...), handle similarly or skip
            continue
        
        cos_obj = cos_objs[0]               # e.g. cos(theta1), cos(... - theta2), etc.
        cterm_factor = e / cos_obj      # factor outside cos(...)
        cos_argument = cos_obj.args[0]      # The argument of cos(...)

        # Lambdify that factor if it depends on n_g or Phi
        cterm_factor_func = sp.lambdify(
            (n1_sym, n2_sym, ng1_sym, ng2_sym, phi_sym),
            cterm_factor, 
            'numpy'
        )

        cos_data_list.append((cterm_factor_func, cos_argument))

    # 6) Build the charge basis: (n1, n2) in [-n_cut..n_cut]
    basis = [(n1, n2) for n1 in range(-n_cut, n_cut+1) for n2 in range(-n_cut, n_cut+1)]
    dim = len(basis)
    n1n2_to_index = {(n1n2): idx for idx, n1n2 in enumerate(basis)}

    # 7) Initialize Hamiltonian matrix
    H = np.zeros((dim, dim), dtype=complex)

    # 8) Diagonal entries from polynomial part
    for idx, (n1, n2) in enumerate(basis):
        H_c = poly_func(n1, n2, n_g1, n_g2)
        H[idx, idx] += H_c

    # 9) Off-diagonal from cos(...) terms
    for idx, (n1, n2) in enumerate(basis):
        for cterm_factor_func, cos_arg in cos_data_list:
            # Evaluate numeric factor at (n1,n2,n_g1,n_g2,2πΦ_1)
            factor_val = cterm_factor_func(n1, n2, n_g1, n_g2, 2*np.pi*Phi_1)
            # cos(...) => 0.5*( e^{i(...)} + e^{-i(...)})
            # We'll parse the argument to see if it's θ1, θ2, (φ - θ2), etc.
            cos_arg_simpl = sp.simplify(cos_arg)

            amp = 0.5 * factor_val
            transitions = []

            if cos_arg_simpl == theta1_sym:
                # cos(theta1): (n1->n1±1), no extra phase
                transitions = [((+1, 0), amp), ((-1, 0), amp)]
            elif cos_arg_simpl == theta2_sym:
                # cos(theta2): (n2->n2±1), no extra phase
                transitions = [((0, +1), amp), ((0, -1), amp)]
            elif cos_arg_simpl == (phi_sym - theta2_sym):
                # cos((2πΦ_1) - θ2) => shift n2 ±1, phase e^{± i(2πΦ_1)}
                transitions = [
                    ((0, +1), amp * np.exp(+1j * 2*np.pi*Phi_1)),
                    ((0, -1), amp * np.exp(-1j * 2*np.pi*Phi_1))
                ]
            elif cos_arg_simpl == (theta1_sym + theta2_sym):
                # cos(theta1 + theta2): (n1->n1±1, n2->n2±1), no extra phase
                transitions = [((+1, +1), amp), ((-1, -1), amp)]
            else:
                # If you expect more forms, add more elif statements here
                pass

            # Update the Hamiltonian with these transitions
            for (dn1, dn2), val in transitions:
                n1p, n2p = n1 + dn1, n2 + dn2
                if -n_cut <= n1p <= n_cut and -n_cut <= n2p <= n_cut:
                    idxp = n1n2_to_index[(n1p, n2p)]
                    H[idx, idxp] += val

    # 10) Make H Hermitian
    H = (H + H.conj().T)/2

    # 11) Diagonalize
    eigvals, eigvecs = np.linalg.eigh(H)
    sort_idx = np.argsort(eigvals)
    eigvals = eigvals[sort_idx].real
    eigvecs = eigvecs[:, sort_idx]

    return eigvals, eigvecs, H

In [29]:


# Suppose you have an Nmon instance
# sym_expr = nmon.sym_hamiltonian   # This is your symbolic Hamiltonian from scqubits.

# Now build & diagonalize the matrix "by hand":
n_cut = 1
Phi_1 = 0.0
n_g1 = 0.0
n_g2 = 0.0

eigs, evecs, H_mat = construct_hamiltonian_from_sym(nmon.sym_hamiltonian,
                                                   n_cut, Phi_1, n_g1, n_g2)

print("Eigenvalues (sorted):", eigs)

TypeError: loop of ufunc does not support argument 0 of type Symbol which has no callable cos method